<a href="https://colab.research.google.com/github/ashmibanerjee/cities-llm-project/blob/main/SFT_Trainer_FT_Gemma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 6.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731

In [2]:
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM
from transformers import AutoModelForCausalLM, AutoTokenizer
from google.colab import userdata

In [3]:
HF_TOKEN = userdata.get("HF_TOKEN")

In [4]:

def get_dataset(dataset_name:str, data_files=None, is_public=True):
    if is_public:
        dataset = load_dataset(dataset_name, split="train")
    else:
        dataset = load_dataset(dataset_name, token=True, data_files=data_files)
    return dataset

In [5]:
def convert_pandas(dataset):
    df = dataset
    df.set_format(type='pandas')  ## converting it into pandas
    try:
        df = df[:]
    except KeyError:
        df = df["train"][:]
    return df

In [6]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['prompt'])):
        text = f"### Question: {example['prompt'][i]}\n ### Answer: {example['answer'][i]}"
        output_texts.append(text)
    return output_texts


In [7]:
dataset = get_dataset(dataset_name="RecSysTUM/europeancities-wikivoyage-tripadvisor", data_files="wikivoyage/q_a/cities_qa_merged.csv", is_public=False)
print(dataset)
df = convert_pandas(dataset)
df.head()

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['city', 'prompt', 'answer'],
        num_rows: 5285
    })
})


,city,prompt,answer
0,Belgrade,What is Belgrade known for?,"Belgrade is known for its vibrant nightlife, h..."
1,Belgrade,What is the history of Belgrade?,Belgrade has a long history dating back to the...
2,Belgrade,What is the climate like in Belgrade?,Belgrade has a temperate continental climate w...
3,Belgrade,What are the people like in Belgrade?,Belgradians are known for their friendliness a...
4,Belgrade,How can I get to Belgrade by plane?,Belgrade Nikola Tesla Airport (BEG) is the mai...


In [8]:
model_name = "google/gemma-2b"
# model_name = "facebook/opt-350m"

model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", token=HF_TOKEN)
tokenizer = AutoTokenizer.from_pretrained(model_name, token=HF_TOKEN)
tokenizer.padding_side = "right"

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [11]:
response_template = " ### Answer:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)
training_data = dataset["train"].train_test_split(test_size=0.3, shuffle=True)

trainer = SFTTrainer(
    model,
    train_dataset=training_data["train"],
    formatting_func=formatting_prompts_func,
    data_collator=collator,
)


/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:146: UserWarning: No `SFTConfig` passed, using `output_dir=tmp_trainer`.
  warnings.warn(f"No `SFTConfig` passed, using `output_dir={output_dir}`.")
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:278: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/3699 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:397: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [ ]:
# trainer.train()
# trainer.save_model()

In [ ]:
#    sft_config = SFTConfig(
#         max_seq_length=512,
#         output_dir="/tmp",
#     )